In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ff68  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 10 12:59 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir-final-2337217' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
inverted_index_body_gcp.py
postings_gcp/0_posting_locs.pickle
postings_gcp/100_posting_locs.pickle
postings_gcp/101_posting_locs.pickle
postings_gcp/102_posting_locs.pickle
postings_gcp/103_posting_locs.pickle
postings_gcp/104_posting_locs.pickle
postings_gcp/105_posting_locs.pickle
postings_gcp/106_posting_locs.pickle
postings_gcp/107_posting_locs.pickle
postings_gcp/108_posting_locs.pickle
postings_gcp/109_posting_locs.pickle
postings_gcp/10_posting_locs.pickle
postings_gcp/110_posting_locs.pickle
postings_gcp/111_posting_locs.pickle
postings_gcp/112_posting_locs.pickle
postings_gcp/113_posting_locs.pickle
postings_gcp/114_posting_locs.pickle
postings_gcp/115_posting_locs.pickle
postings_gcp/116_posting_locs.pickle
postings_gcp/117_posting_locs.pickle
postings_gcp/118_posting_locs.pickle
postings_gcp/119_posting_locs.pickle
postings_gcp/11_posting_locs.pickle
postings_gcp/120_posting_locs.pickle
postings_gcp/121_posting_locs.pickle
postings_gcp/122_posting_locs.pickle


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [17]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_anchor_pairs = parquetFile.select("id", "anchor_text").rdd

In [18]:
# Count number of wiki pages
parquetFile.count()

6348910

In [19]:
import json

with open('/home/dataproc/queries_train.json', 'rt') as f:
  queries = json.load(f)

In [20]:

# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_body_gcp.py

inverted_index_body_gcp.py


In [21]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_body_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

22/01/10 13:04:19 WARN org.apache.spark.SparkContext: The path /home/dataproc/inverted_index_body_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [22]:
from inverted_index_body_gcp import InvertedIndex

In [25]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count_anchor(anchor, original_id):
  pairs_anchor = []
  for item in anchor:
    tokens = [token.group() for token in RE_WORD.finditer(item[1].lower())]
    tokens = [x for x in tokens if x not in all_stopwords]
    pairs = {}
    pairs_tokens = {}
    c = Counter(tokens)
    for token in tokens:
      tf = c[token]
      id = item[0]
      if id in pairs.keys():
        old_tf = pairs[id]
        pairs[id] = old_tf + tf
        pairs_tokens[token] = (id, pairs[id])
      else:
        pairs[id] = tf
        pairs_tokens[token] = (id, tf) 
    pairs_anchor += list(pairs_tokens.items())
  return pairs_anchor

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
 
  sorted_pl = sorted(unsorted_pl)
  return sorted_pl


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  token_df_pairs = postings.map(lambda item: (item[0], len(item[1])))
  return token_df_pairs

def group_by_key(term, posting_list):
  res = OrderedDict()
  for (id, tf) in posting_list:
    if term in res:
      prev_dict = res[term]
      if id in prev_dict.keys():
        prev_tf = prev_dict[id]
        prev_dict[id] = tf + prev_tf
      else:
        prev_dict[id] = tf
    else:
      res[term] = {id : tf}
  for word in res.keys():
    res[word] = list(res[word].items())
  res = list(res.items())
  return res


def partition_postings_and_write(postings, indexNum):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  token_loc_pairs = postings.map(lambda item: (token2bucket_id(item[0]), item))
  buckets = token_loc_pairs.groupByKey().mapValues(list)
  return buckets.map(lambda bucket: InvertedIndex.write_a_posting_list((bucket[0], bucket[1]), bucket_name, indexNum))

In [26]:
# time the index creation time
t_start = time()
# word counts map
word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count_anchor(x[1], x[0]))
postings_anchor_sorted = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
postings_anchor = postings_anchor_sorted.flatMap(lambda item: group_by_key(item[0], item[1]))
del postings_anchor_sorted
# calculate df
w2df_anchor = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor.collectAsMap()
del w2df_anchor
# partition posting lists and write out
posting_locs_list = partition_postings_and_write(postings_anchor, 400).collect()
index_const_time = time() - t_start

In [27]:
# test index construction time
# assert index_const_time < 60*120

In [28]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    

In [29]:
#calculate DL for anchor
def create_anchor_DL(item):
  temp_DL_item = []
  for (id, text) in item[1]:
    elem = [(id, 0)]
    temp_DL_item += elem
  return temp_DL_item

temp_DL_anchor_dict = doc_anchor_pairs.flatMap(lambda item: create_anchor_DL(item))
DL_anchor_dict = temp_DL_anchor_dict.collectAsMap()


In [31]:
#title dict for anchor
def create_title_anchor(item):
  title_anchor_pairs = []
  for (id, text) in item[1]:
    # url = "https://en.wikipedia.org/?curid=" + str(id)
    # html = request.urlopen(url).read().decode('utf8')
    # html[:60]
    # soup = BeautifulSoup(html, 'html.parser')
    # title = soup.find('title')
    try:
      title = title_dict[id]
    except:
      title = '???'
    title_anchor_pairs += [(id, title)]
  return title_anchor_pairs

temp_title_pairs_anchor = doc_anchor_pairs.flatMap(lambda item: create_title_anchor(item))
title_dict_anchor = temp_title_pairs_anchor.collectAsMap() 
del temp_title_pairs_anchor

22/01/10 13:24:24 ERROR org.apache.spark.scheduler.TaskSetManager: Total size of serialized results of 121 tasks (1962.7 MiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
22/01/10 13:24:24 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 75.0 in stage 16.0 (TID 990) (cluster-ff68-w-2.us-central1-a.c.citric-name-337217.internal executor 11): TaskKilled (Tasks result size has exceeded maxResultSize)
22/01/10 13:24:24 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 83.0 in stage 16.0 (TID 998) (cluster-ff68-w-3.us-central1-a.c.citric-name-337217.internal executor 3): TaskKilled (Stage cancelled)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 121 tasks (1962.7 MiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2259)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2208)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2207)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2207)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1079)
	at org.apache.spark

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_anchor_dict
# # Add the DL dictionary to the inverted index
# inverted_anchor.DL = DL_anchor_dict
# Add the title dictionary to the inverted index
inverted_anchor.titles = title_dict_anchor
# write the global stats out
inverted_anchor.write_index('.', 'index_anchor')
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

22/01/10 13:24:24 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 71.0 in stage 16.0 (TID 986) (cluster-ff68-w-3.us-central1-a.c.citric-name-337217.internal executor 4): TaskKilled (Stage cancelled)
22/01/10 13:24:24 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 79.0 in stage 16.0 (TID 994) (cluster-ff68-w-3.us-central1-a.c.citric-name-337217.internal executor 4): TaskKilled (Stage cancelled)
